In [186]:
import sys
sys.path.append('../..')
import pandas as pd
import ta
from utilities.data_manager import ExchangeDataManager
from numpy_ext import rolling_apply as rolling_apply_ext

In [488]:
class Strategy():
    def __init__(
        self,
        pair,
        type=["long"],
        params={},
    ):
        self.df_pair = None
        self.df = None
        self.pair = pair
        self.initial_wallet = 1000
        self.use_long = "long" in type
        self.use_short = "short" in type
        self.params = params
        self.result_df = None

    def get_pair_data(self, timeframe, start = 2050, end = 2050):
        exchange = ExchangeDataManager(
            exchange_name=exchange_name,
            path_download="./database/exchanges"
        )

        self.df_pair = exchange.load_data(self.pair, timeframe, start)

    def populate_indicators2(self):
        params = self.params
        df = self.df_pair.copy()
        df.drop(
            columns=df.columns.difference(['open','high','low','close','volume']),
            inplace=True
        )
        df['fast_ma'] = ta.trend.sma_indicator(close=df["close"], window=params["fast_ma"])
        df['slow_ma'] = ta.trend.sma_indicator(close=df["close"], window=params["slow_ma"])
        df['mrat'] = df['fast_ma'] / df['slow_ma']
        df['mean_mrat'] = ta.trend.sma_indicator(close=df['mrat'], window=params["mean_mrat_lenght"])
        df['stdev_mrat'] = df['mrat'].rolling(params["mean_mrat_lenght"]).std(ddof=0)
        df['open_long_signal'] = df['mean_mrat'].shift(1) - df['mrat'].shift(1) >= params['sigma_open'] * df['stdev_mrat'].shift(1)
        df['close_long_signal'] = df['mrat'].shift(1) - df['mean_mrat'].shift(1) >= params['sigma_close'] * df['stdev_mrat'].shift(1)

        df_signal = df.loc[
            df["open_long_signal"] | df["close_long_signal"],
            ["open_long_signal", "close_long_signal","open", "close"]
        ]
        df_signal["open_signal_lag"] = df_signal["open_long_signal"].shift(fill_value=False)
        df_signal["close_signal_lag"] = df_signal["close_long_signal"].shift(fill_value=False)
        df_first_signal  = df_signal[
            (~ df_signal["open_signal_lag"] & (df_signal["open_long_signal"] | df_signal["open_long_signal"].isnull())) |
            (~ df_signal["close_signal_lag"] & df_signal["close_long_signal"])
         ]
        df_first_signal["open_signal_lag"] = df_first_signal["open_long_signal"].shift(fill_value=False)
        df_first_signal["close_signal_lag"] = df_first_signal["close_long_signal"].shift(fill_value=False)

        df_order_tmp = df_first_signal[
            (df_first_signal["open_long_signal"] & (~df_first_signal["open_signal_lag"] | df_first_signal["open_long_signal"].isnull())) |
            (df_first_signal["close_long_signal"] & ~ df_first_signal["close_signal_lag"])
        ]
        df_order = df_order_tmp.loc[
            ~ ( ~ df_order_tmp["close_signal_lag"] & ~ df_order_tmp["open_signal_lag"] & df_order_tmp["close_long_signal"])
        ]
        df_order["order_number"] = df_order["open_long_signal"].cumsum()
        df_order["open_lag"] = df_order["open"].shift(-1)
        df_order["open_order"] = df_order["open"].shift()
        df_order.loc[df_order["open_long_signal"], "open_order"] = df_order.loc[df_order["open_long_signal"], "open"]
        df_pair = df[["open", "close", "low", "high", "mrat", "mean_mrat", "stdev_mrat", "open_long_signal", "close_long_signal"]]

        leverage = params["leverage"]  # Fixed leverage
        maintenance_margin_percent = 0.004
        wallet = 1000  # Initial wallet balance
        quantity = 0  # Initial quantity

        # Ensure the DataFrame has 'quantity' and 'trade_result' columns initialized
        df_order['quantity'] = 0.0
        df_order['trade_result'] = 0.0
        df_order['trade_result_pct'] = 0.0

        # Iterating over DataFrame rows to process trading signals
        for i, row in df_order.iterrows():
            # Check if there is a signal to open a long position
            if row['open_long_signal']:
                # Calculate the new quantity based on the current wallet and leverage
                quantity = wallet * leverage / row['open']
                # Update the 'quantity' column with the new quantity
                df_order.at[i, 'quantity'] = quantity
                # No change in wallet yet as the position has just opened
                df_order.at[i, 'wallet'] = wallet
                # Track the price at which the position was opened
                open = row['open']
            elif row['close_long_signal']:
                # Calculate the trade result based on the difference between current and open price
                trade_result = (row['open'] - open) * quantity
                # Update the 'trade_result' column with the result of the closed trade
                df_order.at[i, 'trade_result'] = trade_result
                df_order.at[i, 'trade_result_pct'] = trade_result / wallet * 100
                # Update the wallet with the result of the trade
                wallet += trade_result
                # Reset quantity as the trade is closed
                df_order.at[i, 'quantity'] = quantity
                quantity = 0

            # Update the wallet and quantity for the current row
            df_order.at[i, 'wallet'] = wallet

        df_order_tmp = df_order[
            ["order_number", "quantity", "trade_result", "trade_result_pct", "wallet", "open_order"]
        ]
        df_order_final = df_pair.join(df_order_tmp)

        f = df_order_final['order_number'].ffill()
        b = df_order_final['order_number'].bfill()

        df_order_final['order_number'] = df_order_final['order_number'].mask(f == b, f)

        f = df_order_final['open_order'].ffill()
        b = df_order_final['open_order'].bfill()

        df_order_final['open_order'] = df_order_final['open_order'].mask(f == b, f)
        df_order_final['wallet'] = df_order_final['wallet'].ffill()
        #df_order_final["hypothetical_wallet"] = df_order_final["wallet"] + df_order_final["quantity"] * (df_order_final['open'] - df_order_final["open_order"])
        df_order_final["hypothetical_wallet"] = df_order_final["wallet"].shift() + df_order_final["quantity"] * (df_order_final['open'] - df_order_final["open_order"])
        df_order_final["hypothetical_low_result"] =  ((df_order_final["quantity"] * df_order_final["low"]) - df_order_final["wallet"]) / df_order_final["wallet"]
        df_order_final["drawdown"] = (df_order_final["low"] - df_order_final["open_order"]) / df_order_final["open_order"] * 100 * leverage
        df_order_final["is_liquidated"] = df_order_final['hypothetical_wallet'] < (df_order_final["wallet"] / leverage) * maintenance_margin_percent

        return df_order_final


    def populate_indicators(self):
        params = self.params
        df = self.df_pair.copy()
        df.drop(
            columns=df.columns.difference(['open','high','low','close','volume']),
            inplace=True
        )

        # -- Populate indicators --
        df['fast_ma'] = ta.trend.sma_indicator(close=df["close"], window=params["fast_ma"])
        df['slow_ma'] = ta.trend.sma_indicator(close=df["close"], window=params["slow_ma"])
        df['mrat'] = df['fast_ma'] / df['slow_ma']
        df['mean_mrat'] = ta.trend.sma_indicator(close=df['mrat'], window=params["mean_mrat_lenght"])
        df['stdev_mrat'] = df['mrat'].rolling(params["mean_mrat_lenght"]).std(ddof=0)
        df['open_long_signal'] = df['mean_mrat'].shift(1) - df['mrat'].shift(1) >= params['sigma_open'] * df['stdev_mrat'].shift(1)
        df['close_long_signal'] = df['mrat'].shift(1) - df['mean_mrat'].shift(1) >= params['sigma_close'] * df['stdev_mrat'].shift(1)

        df["is_liquidated"] = False
        df["order_open"] = False
        # Trading logic
        order_open = False
        current_order_number = 0
        open_price = 0
        quantity = 0
        trade_result = 0
        # Constants and Initialization
        initial_wallet = self.initial_wallet
        leverage = params["leverage"]  # Fixed leverage
        maintenance_margin_percent = 0.004
        wallet = initial_wallet
        max_equity = initial_wallet  # To track the max equity before a new trade
        max_drawdown = 0

        for i in df.index:
            if df.loc[i, 'open_long_signal'] and not order_open:
                # Open a new order
                current_order_number += 1
                order_open = True
                open_price = df.loc[i, 'open']
                open_wallet = df.loc[i, 'wallet']
                quantity = (wallet / open_price) * leverage
                df.loc[i, 'order_number'] = current_order_number
                df.loc[i, 'order_open'] = order_open

            # Assign order_number to all rows of the current order
            if order_open:
                df.loc[i, 'order_number'] = current_order_number
                df.loc[i, 'order_open'] = order_open
                # Calculate hypothetical_wallet
                hypothetical_wallet = wallet + quantity * (df.loc[i, 'open'] - open_price)
                df.loc[i, 'hypothetical_wallet'] = hypothetical_wallet
                df.loc[i, 'quantity'] = quantity

                # Check for liquidation
                maintenance_margin = (wallet / leverage) * maintenance_margin_percent
                if hypothetical_wallet < maintenance_margin:
                    df.loc[i, 'is_liquidated'] = True
                    df.loc[i, 'trade_result'] = hypothetical_wallet - wallet
                    wallet = 0  # Update wallet with the loss
                    order_open = False  # Close the order

            # Close the order
            if df.loc[i, 'close_long_signal'] and order_open:
                trade_result = quantity * (df.loc[i, 'open'] - open_price)
                trade_result_perc = trade_result / wallet * 100
                wallet += trade_result  # Update wallet with the profit or loss
                order_open = False  # Close the order
                df.loc[i, 'trade_result'] = trade_result
                df.loc[i, 'trade_result_perc'] = trade_result_perc

            # Set wallet to current wallet value
            df.loc[i, 'wallet'] = wallet

        df["drawdown"] = (df["hypothetical_wallet"] - df["wallet"]) / df["wallet"] * 100

        return df


    def get_result_df(self):
        df = self.df
        final_wallet_amount = df.loc[df["order_open"] & df["close_long_signal"], "wallet"].tail(1)
        total_profit = final_wallet_amount  - self.initial_wallet
        total_profit_perc = total_profit / self.initial_wallet * 100
        total_trades = df["order_number"].max()
        avg_trade_profit_perc = df["trade_result_perc"].dropna().mean()
        avg_trade_profit = df["trade_result"].dropna().mean()
        max_drawdown = df["drawdown"].max()

        result_df = pd.DataFrame(
            {
                "params": str(self.params),
                "final_wallet_amount": final_wallet_amount,
                "total_profit": total_profit,
                "total_profit_perc": total_profit_perc,
                "total_trades": total_trades,
                "avg_trade_profit_perc": avg_trade_profit_perc,
                "avg_trade_profit": avg_trade_profit,
                "max_drawdown": max_drawdown,
            }
        )

        self.result_df = result_df

In [475]:
params = {
    "fast_ma": 5,
    "slow_ma": 60,
    "sigma_open": 1,
    "sigma_close": 2.9,
    "mean_mrat_lenght": 60,
    "leverage": 1
}

pair = "API3/USDT:USDT"
exchange_name = "binance"
tf = '15m'
oldest_pair = "API3/USDT:USDT"
start_date = "2023-01-01 00:00:00"

In [476]:
strat = Strategy(pair=pair, params=params)
strat.get_pair_data(timeframe=tf, start=start_date)
df1 = strat.populate_indicators()

In [489]:
strat = Strategy(pair=pair, params=params)
strat.get_pair_data(timeframe=tf, start=start_date)
df2 = strat.populate_indicators2()

                     open_long_signal  close_long_signal    open   close  \
date                                                                       
2023-01-04 02:00:00             False               True  1.0740  1.0750   
2023-01-04 11:15:00              True              False  1.0800  1.0770   
2023-01-08 20:30:00             False               True  1.1560  1.1640   
2023-01-09 11:30:00              True              False  1.2330  1.2330   
2023-01-20 17:45:00             False               True  1.2870  1.2900   
...                               ...                ...     ...     ...   
2024-03-02 10:00:00              True              False  3.8753  3.8697   
2024-03-09 05:00:00             False               True  4.1232  4.2837   
2024-03-09 11:15:00              True              False  3.8996  3.9059   
2024-03-11 00:15:00             False               True  4.0703  3.9796   
2024-03-11 14:00:00              True              False  3.8156  3.8310   

           

/var/folders/_9/mg2v8fwd04n3k0y4zztq8z7h0000gn/T/ipykernel_55277/604151727.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_first_signal["open_signal_lag"] = df_first_signal["open_long_signal"].shift(fill_value=False)
/var/folders/_9/mg2v8fwd04n3k0y4zztq8z7h0000gn/T/ipykernel_55277/604151727.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_first_signal["close_signal_lag"] = df_first_signal["close_long_signal"].shift(fill_value=False)
/var/folders/_9/mg2v8fwd04n3k0y4zztq8z7h0000gn/T/ipykernel_5

UnboundLocalError: local variable 'open' referenced before assignment

In [511]:
df1.order_number.max()

108.0

In [491]:
df1.loc[df1["order_number"] == 1].iloc[[0,-1], :]

,open,high,low,close,volume,fast_ma,slow_ma,mrat,mean_mrat,stdev_mrat,...,close_long_signal,is_liquidated,order_open,wallet,order_number,hypothetical_wallet,quantity,trade_result,trade_result_perc,drawdown
date,,,,,,,,,,,,,,,,,,,,,
2023-01-02 21:45:00,1.053,1.056,1.052,1.053,101779.7,1.0524,1.040250,1.011680,1.016680,0.003903,...,False,False,True,1000.00000,1.0,1000.00000,949.667616,NaN,NaN,0.0
2023-01-04 02:00:00,1.074,1.079,1.074,1.075,117098.9,1.0708,1.042483,1.027163,0.998354,0.009479,...,True,False,True,1019.94302,1.0,1019.94302,949.667616,19.94302,1.994302,0.0


In [458]:
df2.loc[df2["order_number"] == 8].iloc[[0,-1], :]

,open,close,low,high,mrat,mean_mrat,stdev_mrat,open_long_signal,close_long_signal,order_number,quantity,trade_result,trade_result_pct,wallet,open_order,hypothetical_wallet,hypothetical_low_result,drawdown,is_liquidated
date,,,,,,,,,,,,,,,,,,,
2023-01-28 08:30:00,1.619,1.618,1.614,1.620,0.984200,0.999415,0.006731,True,False,8.0,1832.828366,0.000000,0.000000,1483.674563,1.619,1483.674563,0.993823,-0.617665,False
2023-01-29 01:15:00,1.601,1.604,1.594,1.614,1.002999,0.984805,0.005801,False,True,8.0,1832.828366,-32.990911,-2.223595,1450.683652,1.619,1450.683652,1.013898,-3.088326,False


In [483]:
df = df1[["open", "close", "low", "high"]]

In [507]:
df['fast_ma'] = ta.trend.sma_indicator(close=df["close"], window=params["fast_ma"])
df['slow_ma'] = ta.trend.sma_indicator(close=df["close"], window=params["slow_ma"])
df['mrat'] = df['fast_ma'] / df['slow_ma']
df['mean_mrat'] = ta.trend.sma_indicator(close=df['mrat'], window=params["mean_mrat_lenght"])
df['stdev_mrat'] = df['mrat'].rolling(params["mean_mrat_lenght"]).std(ddof=0)
df['open_long_signal'] = df['mean_mrat'].shift(1) - df['mrat'].shift(1) >= params['sigma_open'] * df['stdev_mrat'].shift(1)
df['close_long_signal'] = df['mrat'].shift(1) - df['mean_mrat'].shift(1) >= params['sigma_close'] * df['stdev_mrat'].shift(1)

df_signal = df.loc[
    df["open_long_signal"] | df["close_long_signal"],
    ["open_long_signal", "close_long_signal","open", "close"]
]
df_signal["open_signal_lag"] = df_signal["open_long_signal"].shift(fill_value=False)
df_signal["close_signal_lag"] = df_signal["close_long_signal"].shift(fill_value=False)
df_first_signal  = df_signal[
    (~ df_signal["open_signal_lag"] & (df_signal["open_long_signal"])) |
    (~ df_signal["close_signal_lag"] & df_signal["close_long_signal"])
 ]
df_first_signal["open_signal_lag"] = df_first_signal["open_long_signal"].shift(fill_value=False)
df_first_signal["close_signal_lag"] = df_first_signal["close_long_signal"].shift(fill_value=False)

df_order_tmp = df_first_signal[
    (df_first_signal["open_long_signal"] & (~df_first_signal["open_signal_lag"] | df_first_signal["open_long_signal"].isnull())) |
    (df_first_signal["close_long_signal"] & ~ df_first_signal["close_signal_lag"])
]
df_order = df_order_tmp.loc[~ ( ~ df_order_tmp["close_signal_lag"] & ~ df_order_tmp["open_signal_lag"] & df_order_tmp["close_long_signal"])]
df_order["order_number"] = df_order["open_long_signal"].cumsum()

/var/folders/_9/mg2v8fwd04n3k0y4zztq8z7h0000gn/T/ipykernel_55277/2872500578.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['fast_ma'] = ta.trend.sma_indicator(close=df["close"], window=params["fast_ma"])
/var/folders/_9/mg2v8fwd04n3k0y4zztq8z7h0000gn/T/ipykernel_55277/2872500578.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['slow_ma'] = ta.trend.sma_indicator(close=df["close"], window=params["slow_ma"])
/var/folders/_9/mg2v8fwd04n3k0y4zztq8z7h0000gn/T/ipykernel_55277/2872500578.py:3: Settin

In [508]:
df_order_tmp

,open_long_signal,close_long_signal,open,close,open_signal_lag,close_signal_lag
date,,,,,,
2023-01-02 21:45:00,True,False,1.0530,1.0530,False,False
2023-01-04 02:00:00,False,True,1.0740,1.0750,True,False
2023-01-04 11:15:00,True,False,1.0800,1.0770,False,True
2023-01-08 20:30:00,False,True,1.1560,1.1640,True,False
2023-01-09 11:30:00,True,False,1.2330,1.2330,False,True
...,...,...,...,...,...,...
2024-03-02 10:00:00,True,False,3.8753,3.8697,False,True
2024-03-09 05:00:00,False,True,4.1232,4.2837,True,False
2024-03-09 11:15:00,True,False,3.8996,3.9059,False,True


In [510]:
df_order.order_number.max()

108

In [485]:
df_order["open_lag"] = df_order["open"].shift(-1)
df_order["open_order"] = df_order["open"].shift()
df_order.loc[df_order["open_long_signal"], "open_order"] = df_order.loc[df_order["open_long_signal"], "open"]
df_pair = df[["open", "close", "low", "high", "mrat", "mean_mrat", "stdev_mrat", "open_long_signal", "close_long_signal"]]

leverage = params["leverage"]  # Fixed leverage
maintenance_margin_percent = 0.004
wallet = 1000  # Initial wallet balance
quantity = 0  # Initial quantity

# Ensure the DataFrame has 'quantity' and 'trade_result' columns initialized
df_order['quantity'] = 0.0
df_order['trade_result'] = 0.0
df_order['trade_result_pct'] = 0.0

# Iterating over DataFrame rows to process trading signals
for i, row in df_order.iterrows():
    # Check if there is a signal to open a long position
    if row['open_long_signal']:
        # Calculate the new quantity based on the current wallet and leverage
        quantity = wallet * leverage / row['open']
        # Update the 'quantity' column with the new quantity
        df_order.at[i, 'quantity'] = quantity
        # No change in wallet yet as the position has just opened
        df_order.at[i, 'wallet'] = wallet
        # Track the price at which the position was opened
        open = row['open']
    elif row['close_long_signal']:
        # Calculate the trade result based on the difference between current and open price
        trade_result = (row['open'] - open) * quantity
        # Update the 'trade_result' column with the result of the closed trade
        df_order.at[i, 'trade_result'] = trade_result
        df_order.at[i, 'trade_result_pct'] = trade_result / wallet * 100
        # Update the wallet with the result of the trade
        wallet += trade_result
        # Reset quantity as the trade is closed
        df_order.at[i, 'quantity'] = quantity
        quantity = 0

    # Update the wallet and quantity for the current row
    df_order.at[i, 'wallet'] = wallet

/var/folders/_9/mg2v8fwd04n3k0y4zztq8z7h0000gn/T/ipykernel_55277/4262159250.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_order["open_lag"] = df_order["open"].shift(-1)
/var/folders/_9/mg2v8fwd04n3k0y4zztq8z7h0000gn/T/ipykernel_55277/4262159250.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_order["open_order"] = df_order["open"].shift()
/var/folders/_9/mg2v8fwd04n3k0y4zztq8z7h0000gn/T/ipykernel_55277/4262159250.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice